In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lmfit.models

#Rutherfordstreuung

##Rutherfordsteuuformel
<br>
$\huge \huge \frac{d\sigma}{d\Omega} = 1.3 \cdot 10^{-3} \left( \frac{Z_1 Z_2}{E_0} \right) \frac{1}{sin^4\left(\frac{\theta}{2}\right)}$

<ul>
    <li>$\sigma$: Differentielle WQ
    <li>$\Theta$: Winkel
    <li>$Z_i$: Kernlagundszahl
    <li>E: Energie
</ul>   

In [11]:
def rutherfordsteuung(theta,Z_1,Z_2,E):
    e = 1.602 * 10**-19
    return (Z_1*Z_2*e**2)/(4*E*np.sin(theta/2)**4)

In [12]:
x = np.arange(-180,180)

In [15]:
plt.plot(x,rutherfordsteuung(x,100,4,100))
plt.show()